In [87]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers

In [88]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [89]:
directory_path = r'/home/ryan/Documents/CBI/nesscan_fixed_removed_flat_lines_03312025/bobHallPier'
dataframes = []
for filename in sorted(os.listdir(directory_path)):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(
            file_path,
            parse_dates=[0],
            na_values=[-999, -99, 99, 'NA', 'RM'],
            engine='python'
        )
        df.columns = ['date', 'pwl']
        dataframes.append(df)

bhp = pd.concat(dataframes, ignore_index=True)
bhp['date'] = pd.to_datetime(bhp['date'])
bhp = bhp[bhp['date'] >= pd.to_datetime('1996-01-01')]
bhp = bhp[bhp['date'] <= pd.to_datetime('2008-01-01')]
bhp = bhp.reset_index(drop=True)

bhp

,date,pwl
0,1996-01-01 00:00:00,6.800
1,1996-01-01 00:06:00,6.773
2,1996-01-01 00:12:00,6.806
3,1996-01-01 00:18:00,6.833
4,1996-01-01 00:24:00,6.869
...,...,...
1051916,2007-12-31 23:36:00,6.708
1051917,2007-12-31 23:42:00,6.723
1051918,2007-12-31 23:48:00,6.717
1051919,2007-12-31 23:54:00,6.723


In [90]:
pc = pd.read_csv('/home/ryan/Downloads/pd_1747414029.csv')
pc['date'] = pd.to_datetime(pc['#date+time'],errors='coerce')
pc = pc.drop(columns=['#date+time'])
pc = pc.drop(pc.index[-2:])
pc

/tmp/ipykernel_6304/1465235489.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  pc = pd.read_csv('/home/ryan/Downloads/pd_1747414029.csv')


,005-pwl,date
0,0.910,1996-01-01 00:00:00
1,0.911,1996-01-01 00:06:00
2,0.913,1996-01-01 00:12:00
3,0.914,1996-01-01 00:18:00
4,0.916,1996-01-01 00:24:00
...,...,...
1052155,0.947,2008-01-01 23:30:00
1052156,0.945,2008-01-01 23:36:00
1052157,0.946,2008-01-01 23:42:00
1052158,0.945,2008-01-01 23:48:00


In [91]:
main = bhp.merge(pc, on='date', how='inner')
main['005-pwl'] = pd.to_numeric(main['005-pwl'], errors='coerce')

In [92]:
STEP_MIN      = 6          # native resolution of the data
LOOKBACK_HR   = 4          # hours of history you want
LOOKBACK_ROWS = LOOKBACK_HR * 60 // STEP_MIN
df_filled = main.ffill(limit=LOOKBACK_ROWS)

In [95]:
good = (~np.isnan(X).any(axis=1)) & (~np.isinf(X).any(axis=1))
X, y  = X[good], y[good]
print("After finite-row filter:", X.shape, y.shape)   # sanity check

# ───────────────────────────────────────────────────────────────
# 2️⃣  Train/val split (time-ordered) & scaling
# ───────────────────────────────────────────────────────────────
split = int(len(X) * 0.8)
X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]

sx = StandardScaler().fit(X_train)
sy = StandardScaler().fit(y_train.reshape(-1, 1))

X_train = sx.transform(X_train)
X_val   = sx.transform(X_val)
y_train = sy.transform(y_train.reshape(-1, 1)).ravel()
y_val   = sy.transform(y_val.reshape(-1, 1)).ravel()

# ───────────────────────────────────────────────────────────────
# 3️⃣  REPLACE any residual NaN/Inf after scaling (paranoia guard)
# ───────────────────────────────────────────────────────────────
X_train = np.nan_to_num(X_train, nan=0.0, posinf=0.0, neginf=0.0)
X_val   = np.nan_to_num(X_val,   nan=0.0, posinf=0.0, neginf=0.0)

# ───────────────────────────────────────────────────────────────
# 4️⃣  Lower the learning rate a notch (optional safe-guard)
# ───────────────────────────────────────────────────────────────
from tensorflow import keras
from tensorflow.keras import layers

inp = keras.Input(shape=(81,))
x   = layers.Dense(128, activation='relu')(inp)
x   = layers.Dropout(0.2)(x)
x   = layers.Dense(64,  activation='relu')(x)
x   = layers.Dropout(0.1)(x)
out = layers.Dense(1)(x)

model = keras.Model(inp, out)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=5e-4),  # ↓ from 1e-3
    loss='mae',
    metrics=['mae']
)

hist = model.fit(
    X_train, y_train,
    epochs=200, batch_size=1024,
    validation_data=(X_val, y_val),
    callbacks=[keras.callbacks.EarlyStopping(patience=150,
                                             restore_best_weights=True)],
    verbose=2
)

After finite-row filter: (65254, 81) (65254,)
Epoch 1/200
51/51 - 3s - 58ms/step - loss: 0.7863 - mae: 0.7863 - val_loss: 0.9700 - val_mae: 0.9700
Epoch 2/200
51/51 - 0s - 3ms/step - loss: 0.7502 - mae: 0.7502 - val_loss: 0.9264 - val_mae: 0.9264
Epoch 3/200
51/51 - 0s - 3ms/step - loss: 0.7413 - mae: 0.7413 - val_loss: 0.9135 - val_mae: 0.9135
Epoch 4/200
51/51 - 0s - 3ms/step - loss: 0.7378 - mae: 0.7378 - val_loss: 0.9475 - val_mae: 0.9475
Epoch 5/200
51/51 - 0s - 3ms/step - loss: 0.7335 - mae: 0.7335 - val_loss: 0.9505 - val_mae: 0.9505
Epoch 6/200
51/51 - 0s - 3ms/step - loss: 0.7312 - mae: 0.7312 - val_loss: 0.9214 - val_mae: 0.9214
Epoch 7/200
51/51 - 0s - 3ms/step - loss: 0.7291 - mae: 0.7291 - val_loss: 0.9506 - val_mae: 0.9506
Epoch 8/200
51/51 - 0s - 3ms/step - loss: 0.7265 - mae: 0.7265 - val_loss: 0.9583 - val_mae: 0.9583
Epoch 9/200
51/51 - 0s - 3ms/step - loss: 0.7260 - mae: 0.7260 - val_loss: 0.9613 - val_mae: 0.9613
Epoch 10/200
51/51 - 0s - 3ms/step - loss: 0.7245 - m